# H1 项目的构成和构建

## 项目元素

1. **源代码文件**

    源代码：用编程语言（如c++，c）写的人类可读的指令文本文件

    1. 头文件（如.hpp、.h）
  
        声明函数签名、类结构（告诉编译器“某个东西存在”），但不含具体实现。
      
    3. 源文件（如.cpp、.c）
  
        定义（具体实现）函数、类成员函数、全局变量等，实现程序逻辑

2. **库**

    一堆预先编写好、编译好的、可以直接调用的代码集合，提供常用、复杂的功能，避免你“重复造轮子”（实现代码复用）

    1. 静态库和动态库
  
       
  
    2. 标准库、第三方库和自己写的库
| 类型           | 标准库                                   | 第三方库                                       | 自己写的库                       |
|----------------|----------------------------------------|---------------------------------------------|--------------------------------|
| **谁提供的**    | 语言官方（C++标准委员会）                | 其他开发者/公司（如Google的protobuf）          | 你自己                          |
| **包含什么**    | 基础功能（字符串处理、文件操作等）        | 特定领域功能（网络请求、图形绘制等）            | 项目中重复使用的模块             |
| **如何引用**    | `#include <iostream>`                   | `#include <openssl/md5.h>`                   | `#include "mylib/utils.h"`     |
| **是否需要额外操作** | 直接使用（编译器自带）                  | 需下载安装并配置链接                          | 只需正确包含头文件               |
| **例子**       | `vector`, `string`, `fstream`           | Boost, OpenCV, Qt                            | 自己封装的日志工具、数学工具      |




3. **编译与链接**

    